DOWNLOAD ALL NECESSARY FILE FROM THE FOLDER

In [1]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl
import ipywidgets as widgets
from mpl_toolkits import mplot3d

In [2]:
texts1 = [[], []]
texts2 = [[], []]

with open("Print_09_v2_PCG_LV.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts1[0].append(float(text[0]))
    texts1[1].append(float(text[1]))

with open("Print_13_v2_PCG_RV.txt", "r") as file:
  textfile = file.readlines()

for text in textfile:
    text = text.split('\t')
    text[1] = text[1].replace('\n',"")
    texts2[0].append(float(text[0]))
    texts2[1].append(float(text[1]))


fig = plt.figure()
ax0 = fig.add_axes([0,0.1,0.9,0.35], xlabel="sequence", ylabel="amplitude", title="ElectroCardioGram Signal")
ax1 = fig.add_axes([0,0.60,0.9,0.35], xlabel="sequence", ylabel="amplitude", title="PhonoCardioGram Signal")
ax0.plot(texts1[0], texts1[1])
ax1.plot(texts2[0], texts2[1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
scale = 200
delta = 0.2
dt = 0.001 # 1000 Hz Sampling
coef, freqs=pywt.cwt(texts1[1], np.arange(1, scale, 4), 'morl')
X = texts1[0]
Y = np.linspace(0, np.shape(freqs)[0], np.shape(coef)[0])
Z = abs(coef)
X, Y = np.meshgrid(X, Y)

# fig = plt.figure()
# plt.imshow(coef, interpolation="nearest", origin="upper",aspect='auto')
# plt.colorbar()
# plt.show()
# plt.matshow(coef) # doctest: +SKIP
# plt.show() # doctest: +SKIP

In [12]:
fig = plt.figure()
ax = plt.axes(projection='3d', title="Continuous Wavelet Transform")
plot_cwt = ax.plot_surface(X, Y, Z,rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')
# plot_cwt = ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)

fig.colorbar(plot_cwt, shrink=0.5, aspect=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …